In [ ]:
import music21
import pathlib

from thesession.converter import ABCMusicConverter

In [ ]:
ABCMusicConverter("cooleys.abc", "cooleys").to_mp3(
    instrument="flute",
    tempo=190,
    max_notes=300,
    cut_silence=30,
    start=0.57,
    noise_amplitude=0.001,
    duration=120,
    vbr=8,
    clean_files=True
)

# CLAP

In [ ]:
import torch
from laion_clap import CLAP_Module

# Load pretrained CLAP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLAP_Module(enable_fusion=True) # For using fusion to manage >10s clips
model.load_ckpt()  # downloads pretrained weights
model = model.to(device)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [11]:
import torchaudio

waveform, sr = torchaudio.load("cooleys.mp3")

In [12]:
import torchaudio
import torch


def load_audio(filepath, target_sr=16000):
    waveform, sr = torchaudio.load(filepath)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    return waveform.mean(dim=0).unsqueeze(0)  # Convert to mono, add batch dim

In [13]:
audio_tensor0 = load_audio("audio/1_cooleys/36371_0.mp3")
audio_tensor1 = load_audio("audio/1_cooleys/36371_1.mp3")
audio_tensor2 = load_audio("audio/8_the_banshee/32846_0.mp3")
audio_tensor3 = load_audio("audio/8_the_banshee/32846_1.mp3")

In [14]:
with torch.no_grad():
    embedding0 = model.get_audio_embedding_from_data(audio_tensor0, use_tensor=True)
    embedding1 = model.get_audio_embedding_from_data(audio_tensor1, use_tensor=True)
    embedding2 = model.get_audio_embedding_from_data(audio_tensor2, use_tensor=True)
    embedding3 = model.get_audio_embedding_from_data(audio_tensor3, use_tensor=True)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


In [15]:
embeddings = torch.cat([embedding0, embedding1, embedding2, embedding3], dim=0)
embeddings.shape

torch.Size([4, 512])

In [16]:
import torch.nn.functional as F

F.cosine_similarity(embeddings[None, :, :], embeddings[:, None, :], dim=-1)

tensor([[1.0000, 0.8502, 0.8558, 0.8473],
        [0.8502, 1.0000, 0.9414, 0.8412],
        [0.8558, 0.9414, 1.0000, 0.8524],
        [0.8473, 0.8412, 0.8524, 1.0000]], device='cuda:0')

# Dataset

In [1]:
import pathlib
import numpy as np
import pandas as pd

from thesession.dataset import TheSessionDataset

path = pathlib.Path("audio_flac")
tunes = [p.stem for p in path.iterdir() if p.is_dir()]
tunes.sort()

tunes_df = pd.DataFrame({"tune": tunes.copy(), "dataset": None})

prng = np.random.default_rng(42)
tunes = prng.permutation(tunes)

tunes_df.loc[tunes_df["tune"].isin(tunes[0 : int(0.15 * len(tunes))]), "dataset"] = "test"
tunes_df.loc[tunes_df["tune"].isin(tunes[int(0.15 * len(tunes)) : int(0.3 * len(tunes))]), "dataset"] = "validation"
tunes_df.loc[tunes_df["tune"].isin(tunes[int(0.3 * len(tunes)) :]), "dataset"] = "train"


tunes_df.to_csv("dataset.csv")

In [2]:
test_subset = tunes_df.loc[tunes_df["dataset"] == "test", "tune"]
val_subset = tunes_df.loc[tunes_df["dataset"] == "validation", "tune"]
train_subset = tunes_df.loc[tunes_df["dataset"] == "train", "tune"]

train_dataset = TheSessionDataset("audio_flac", subset=train_subset, sampling_rate=48000, fmt=".flac")
val_dataset = TheSessionDataset("audio_flac", subset=val_subset, sampling_rate=48000, fmt=".flac")
test_dataset = TheSessionDataset("audio_flac", subset=test_subset, sampling_rate=48000, fmt=".flac")

print("Training data", len(train_dataset))
print("Validation data", len(val_dataset))
print("Test data", len(test_dataset))

Training data 26344
Validation data 5534
Test data 5728


In [3]:
train_dataset[32]

(tensor([-0.0138, -0.0157, -0.0151,  ..., -0.0132, -0.0090, -0.0040]),
 tensor([ 0.0086,  0.0108,  0.0123,  ..., -0.0052, -0.0035, -0.0016]))

In [4]:
train_dataset[32][0].max()

tensor(0.0609)

In [5]:
train_dataset[32][0].min()

tensor(-0.0386)

In [6]:
%%timeit

train_dataset[0]

47 ms ± 3.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
import torch

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

In [8]:
import time

start = time.perf_counter()

for i, (x1, x2) in enumerate(train_loader):
    if i < 20:
        print(i, end="\r")
    else:
        break

end = time.perf_counter()
print("Duration: ", end - start)



Duration:  8.030320544996357


# Model

In [1]:
from thesession.model import TheSessionModel

model = TheSessionModel()
model.load_ckpt()

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [2]:
model.toggle_gradients(False)

Disabling gradient for parameter clap_model.model.logit_scale_a
Disabling gradient for parameter clap_model.model.logit_scale_t
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_real.weight
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_imag.weight
Disabling gradient for parameter clap_model.model.audio_branch.logmel_extractor.melW
Disabling gradient for parameter clap_model.model.audio_branch.bn0.weight
Disabling gradient for parameter clap_model.model.audio_branch.bn0.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.mel_conv

In [3]:
model.toggle_gradients(True, ['clap_model.model.audio_projection'])

Enabling gradient for parameter clap_model.model.audio_projection.0.weight
Enabling gradient for parameter clap_model.model.audio_projection.0.bias
Enabling gradient for parameter clap_model.model.audio_projection.2.weight
Enabling gradient for parameter clap_model.model.audio_projection.2.bias


# Entraînement

In [11]:
import pandas as pd

from thesession.dataset import TheSessionDataset
from thesession.model import TheSessionModel
from thesession.training import eval_model, NTXentLoss

# Dataset
dataset = pd.read_csv("dataset.csv")

test_subset = dataset.loc[dataset["dataset"] == "test", "tune"]

test_dataset = TheSessionDataset(
    "audio_flac", subset=test_subset, sampling_rate=48000, fmt=".flac"
)

model = TheSessionModel(device="cuda")
criterion = NTXentLoss(temperature=0.05)
model.eval()

with open("testing.csv", "w") as f:
    f.write("\nstep,test_loss,test_accuracy")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Baseline
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nbaseline,{loss},{acc}")

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


Duration of evaluation on test data: 79.15 seconds
Test Loss: 4.1314 | Test Accuracy: 0.0402 | 


In [13]:
# Pretrained weights
model.load_ckpt()
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\npretrain,{loss},{acc}")

Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


Duration of evaluation on test data: 78.15 seconds
Test Loss: 5.5868 | Test Accuracy: 0.0102 | 


In [17]:
model.load("models/step1_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep1,{loss},{acc}")


Duration of evaluation on test data: 76.79 seconds
Test Loss: 1.3755 | Test Accuracy: 0.8455 | 


In [18]:
model.load("models/step2_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep2,{loss},{acc}")

Duration of evaluation on test data: 77.31 seconds
Test Loss: 1.2046 | Test Accuracy: 0.8943 | 


In [19]:
model.load("models/step3_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep3,{loss},{acc}")

Duration of evaluation on test data: 78.79 seconds
Test Loss: 0.3282 | Test Accuracy: 0.9010 | 


In [20]:
model.load("models/step4_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep4,{loss},{acc}")

Duration of evaluation on test data: 78.08 seconds
Test Loss: 0.1179 | Test Accuracy: 0.9696 | 


In [21]:
model.load("models/step5_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep5,{loss},{acc}")

Duration of evaluation on test data: 78.67 seconds
Test Loss: 0.1029 | Test Accuracy: 0.9763 | 


In [22]:
model.load("models/step6_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep6,{loss},{acc}")

Duration of evaluation on test data: 76.33 seconds
Test Loss: 0.0330 | Test Accuracy: 0.9923 | 


# Retriever

In [1]:
import os
import dotenv

import pandas as pd

from thesession.retriever import TheSessionRetriever, get_database_url

dotenv.load_dotenv("thesession-db/.env")

url = get_database_url(
    os.getenv("POSTGRES_USER"),
    os.getenv("POSTGRES_PASSWORD"),
    port=15432,
    database=os.getenv("POSTGRES_DB")
)

retriever = TheSessionRetriever(url, "models/step6_best.pt", backend="soundfile")

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
retriever("audio_flac/1_cooleys/36371_0.flac", limit=10, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,1,36379,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 9\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.944528
1,1,36384,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 14\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.942206
2,1,36371,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 1\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.938617
3,1,36383,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 13\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.925565
4,1,36389,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 19\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.920079
5,17758,36231,Fiddler Leahy’s,None,https://thesession.org/tunes/17758,reel,1,X: 1\nT: Fiddler Leahy's\nR: reel\nM: 4/4\nL: ...,0.913221
6,1,36386,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 16\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.910002
7,1,36373,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 3\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.907785
8,485,59924,The Glentaun,None,https://thesession.org/tunes/485,reel,191,X: 2\nT: The Glentaun\nR: reel\nM: 4/4\nL: 1/8...,0.903914
9,1,36394,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 24\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.903227


In [3]:
retriever("audio_flac/8_the_banshee/32850_1.flac", limit=10, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,8,32850,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 5\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.957744
1,8,32864,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 19\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8...,0.880581
2,8,32849,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 4\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.837869
3,3484,34596,Adele’s Bath,Jacob Fournel,https://thesession.org/tunes/3484,jig,16,X: 1\nT: Adele's Bath\nR: jig\nM: 6/8\nL: 1/8\...,0.832479
4,8,32858,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 13\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8...,0.828592
5,24360,46018,The Rabbit And The Lamb,Crisdean MacDonald,https://thesession.org/tunes/24360,jig,7,X: 2\nT: The Rabbit And The Lamb\nR: jig\nM: 6...,0.828278
6,21820,17475,Miss Kelly’,None,https://thesession.org/tunes/21820,reel,2,X: 1\nT: Miss Kelly'\nR: reel\nM: 4/4\nL: 1/8\...,0.820788
7,8,32846,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 1\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.818012
8,515,27675,The Rolling Waves,None,https://thesession.org/tunes/515,jig,925,X: 7\nT: The Rolling Waves\nR: jig\nM: 6/8\nL:...,0.814068
9,17942,55451,John Joe Moroney’s Favourite,Dermot Lernihan,https://thesession.org/tunes/17942,reel,6,X: 1\nT: John Joe Moroney's Favourite\nR: reel...,0.801404


In [4]:
retriever("audio_flac/7346_hommage__edmond_parizeau/3724_4.flac", limit=10, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,7346,3724,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 4\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.926649
1,7346,3725,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 5\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.915490
2,7346,3721,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 1\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.914071
3,7346,3723,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 3\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.860482
4,7346,3722,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 2\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.855440
5,7937,13807,Jota Da Maia,None,https://thesession.org/tunes/7937,waltz,23,X: 1\nT: Jota Da Maia\nR: waltz\nM: 3/4\nL: 1/...,0.827008
6,7937,13808,Jota Da Maia,None,https://thesession.org/tunes/7937,waltz,23,X: 2\nT: Jota Da Maia\nR: waltz\nM: 3/4\nL: 1/...,0.810844
7,14494,46477,L’Antre,Fred Guichen,https://thesession.org/tunes/14494,waltz,14,X: 2\nT: L'Antre\nR: waltz\nM: 3/4\nL: 1/8\nK:...,0.790134
8,20083,43476,Bridal Festival Quadrille,None,https://thesession.org/tunes/20083,jig,8,X: 2\nT: Bridal Festival Quadrille\nR: jig\nM:...,0.788990
9,5732,38284,Sir Sidney Smith’s,James Hook,https://thesession.org/tunes/5732,march,81,X: 1\nT: Sir Sidney Smith's\nR: march\nM: 4/4\...,0.785859


In [5]:
retriever("audio_flac/64_the_maid_behind_the_bar/40526_3.flac", limit=10, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,64,40526,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 2\nT: The Maid Behind The Bar\nR: reel\nM: ...,0.975198
1,64,40534,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 10\nT: The Maid Behind The Bar\nR: reel\nM:...,0.957901
2,64,40535,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 11\nT: The Maid Behind The Bar\nR: reel\nM:...,0.945629
3,64,40543,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 19\nT: The Maid Behind The Bar\nR: reel\nM:...,0.934551
4,64,40536,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 12\nT: The Maid Behind The Bar\nR: reel\nM:...,0.917324
5,64,40525,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 1\nT: The Maid Behind The Bar\nR: reel\nM: ...,0.895568
6,64,40542,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 18\nT: The Maid Behind The Bar\nR: reel\nM:...,0.888667
7,64,40532,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 8\nT: The Maid Behind The Bar\nR: reel\nM: ...,0.876188
8,64,40530,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 6\nT: The Maid Behind The Bar\nR: reel\nM: ...,0.860519
9,64,40539,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 15\nT: The Maid Behind The Bar\nR: reel\nM:...,0.845718


In [18]:
retriever("test/banshee2.mp3", limit=10, duration=60, skip_start=5)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,8,32860,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 15\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8...,0.820622
1,8,32859,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 14\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8...,0.815643
2,6812,34069,Maureen Kennelly’s,Charlie McKerron,https://thesession.org/tunes/6812,reel,18,X: 1\nT: Maureen Kennelly's\nR: reel\nM: 4/4\n...,0.797905
3,15726,48277,Taibhse Chonaill,None,https://thesession.org/tunes/15726,jig,24,X: 1\nT: Taibhse Chonaill\nR: jig\nM: 6/8\nL: ...,0.797638
4,11426,18285,You Can Call Me Big Al,Ian Stephenson,https://thesession.org/tunes/11426,jig,58,X: 2\nT: You Can Call Me Big Al\nR: jig\nM: 6/...,0.794724
5,13427,12682,The Knocknagree,Johnny O’Leary,https://thesession.org/tunes/13427,polka,15,X: 1\nT: The Knocknagree\nR: polka\nM: 2/4\nL:...,0.784587
6,1392,48715,The Rollicking Boys Of Tandragee,None,https://thesession.org/tunes/1392,jig,410,X: 14\nT: The Rollicking Boys Of Tandragee\nR:...,0.782814
7,3338,32580,Caha Mountains,John Dwyer,https://thesession.org/tunes/3338,reel,101,X: 5\nT: Caha Mountains\nR: reel\nM: 4/4\nL: 1...,0.762976
8,989,36162,Cloch Na Ceithre Mhile,Arthur Darley,https://thesession.org/tunes/989,reel,175,X: 5\nT: Cloch Na Ceithre Mhile\nR: reel\nM: 4...,0.759100
9,8,32866,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 21\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8...,0.755833


In [22]:
retriever("test/cooleys.mp3", limit=10, duration=60, skip_start=5)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,6812,34069,Maureen Kennelly’s,Charlie McKerron,https://thesession.org/tunes/6812,reel,18,X: 1\nT: Maureen Kennelly's\nR: reel\nM: 4/4\n...,0.871394
1,1,36384,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 14\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.792302
2,3779,60780,Miss McDonald,None,https://thesession.org/tunes/3779,reel,93,X: 2\nT: Miss McDonald\nR: reel\nM: 4/4\nL: 1/...,0.792043
3,1,36389,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 19\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.769709
4,1,36392,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 22\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.765302
5,14482,3073,Arthur Muise,Brenda Stubbert,https://thesession.org/tunes/14482,reel,7,X: 1\nT: Arthur Muise\nR: reel\nM: 4/4\nL: 1/8...,0.760786
6,406,36248,Lad O’Beirne’s,Lad O’Beirne,https://thesession.org/tunes/406,reel,555,X: 2\nT: Lad O'Beirne's\nR: reel\nM: 4/4\nL: 1...,0.755759
7,1,36371,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 1\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.751313
8,1,36376,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 6\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.746507
9,406,36255,Lad O’Beirne’s,Lad O’Beirne,https://thesession.org/tunes/406,reel,555,X: 9\nT: Lad O'Beirne's\nR: reel\nM: 4/4\nL: 1...,0.744546


In [23]:
retriever("test/hommage_edmond_parizeau.mp3", limit=10, duration=60, skip_start=5)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,7346,3724,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 4\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.835859
1,7346,3725,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 5\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.811033
2,14315,50326,Katyusha,None,https://thesession.org/tunes/14315,polka,13,X: 1\nT: Katyusha\nR: polka\nM: 2/4\nL: 1/8\nK...,0.797435
3,7346,3722,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 2\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.786967
4,7346,3721,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 1\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.784508
5,23557,35946,The Immutable A,Paul Sinclair,https://thesession.org/tunes/23557,reel,3,X: 1\nT: The Immutable A\nR: reel\nM: 4/4\nL: ...,0.774189
6,7346,3723,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 3\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.749035
7,20083,43475,Bridal Festival Quadrille,None,https://thesession.org/tunes/20083,jig,8,X: 1\nT: Bridal Festival Quadrille\nR: jig\nM:...,0.738584
8,79,43290,Paddy Ryan’s Dream,None,https://thesession.org/tunes/79,reel,656,X: 5\nT: Paddy Ryan's Dream\nR: reel\nM: 4/4\n...,0.724732
9,11802,1958,Princess Polly,None,https://thesession.org/tunes/11802,jig,17,X: 1\nT: Princess Polly\nR: jig\nM: 6/8\nL: 1/...,0.721925


In [27]:
retriever("test/international_appalachian_trail.mp3", limit=20, duration=90, skip_start=3)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,4997,47396,Ashokan Farewell,Jay Ungar,https://thesession.org/tunes/4997,waltz,1608,X: 2\nT: Ashokan Farewell\nR: waltz\nM: 3/4\nL...,0.840904
1,16722,2489,The Bacup Coconut Tune,None,https://thesession.org/tunes/16722,reel,4,X: 1\nT: The Bacup Coconut Tune\nR: reel\nM: 4...,0.840380
2,15228,39473,Kenmay House,None,https://thesession.org/tunes/15228,barndance,5,X: 1\nT: Kenmay House\nR: barndance\nM: 4/4\nL...,0.821002
3,6999,30870,Archibald McDonald Of Keppoch,None,https://thesession.org/tunes/6999,waltz,274,X: 3\nT: Archibald McDonald Of Keppoch\nR: wal...,0.813352
4,4997,47395,Ashokan Farewell,Jay Ungar,https://thesession.org/tunes/4997,waltz,1608,X: 1\nT: Ashokan Farewell\nR: waltz\nM: 3/4\nL...,0.809195
5,10649,50629,Ar Lan Y Mor,None,https://thesession.org/tunes/10649,barndance,7,X: 1\nT: Ar Lan Y Mor\nR: barndance\nM: 4/4\nL...,0.809015
6,13141,28630,The White Rose,None,https://thesession.org/tunes/13141,waltz,15,X: 1\nT: The White Rose\nR: waltz\nM: 3/4\nL: ...,0.805490
7,23307,55305,I Once Loved A Lady,None,https://thesession.org/tunes/23307,waltz,5,X: 1\nT: I Once Loved A Lady\nR: waltz\nM: 3/4...,0.805028
8,1594,3096,As I Roved Out,None,https://thesession.org/tunes/1594,polka,171,X: 2\nT: As I Roved Out\nR: polka\nM: 2/4\nL: ...,0.802921
9,10481,22218,I Will Go Home To Kintail,None,https://thesession.org/tunes/10481,waltz,74,X: 1\nT: I Will Go Home To Kintail\nR: waltz\n...,0.800960


In [30]:
retriever("test/cat_kitten.mp3", limit=10, duration=60, skip_start=5)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,173,58398,Miss Lyall,None,https://thesession.org/tunes/173,strathspey,215,X: 5\nT: Miss Lyall\nR: strathspey\nM: 4/4\nL:...,0.970031
1,173,58399,Miss Lyall,None,https://thesession.org/tunes/173,strathspey,215,X: 6\nT: Miss Lyall\nR: strathspey\nM: 4/4\nL:...,0.926024
2,173,58394,Miss Lyall,None,https://thesession.org/tunes/173,strathspey,215,X: 1\nT: Miss Lyall\nR: strathspey\nM: 4/4\nL:...,0.904188
3,173,58397,Miss Lyall,None,https://thesession.org/tunes/173,strathspey,215,X: 4\nT: Miss Lyall\nR: strathspey\nM: 4/4\nL:...,0.899142
4,19939,48246,Three Men On A White Horse,None,https://thesession.org/tunes/19939,reel,3,X: 3\nT: Three Men On A White Horse\nR: reel\n...,0.882593
5,24985,57005,The April Child,Annbjørg Lien,https://thesession.org/tunes/24985,waltz,5,X: 1\nT: The April Child\nR: waltz\nM: 3/4\nL:...,0.866820
6,19809,56055,The Piper’s Turn,Charlie Lennon,https://thesession.org/tunes/19809,jig,7,X: 2\nT: The Piper's Turn\nR: jig\nM: 6/8\nL: ...,0.855020
7,173,58395,Miss Lyall,None,https://thesession.org/tunes/173,strathspey,215,X: 2\nT: Miss Lyall\nR: strathspey\nM: 4/4\nL:...,0.854309
8,10065,27861,Valse Astrid,None,https://thesession.org/tunes/10065,waltz,38,X: 2\nT: Valse Astrid\nR: waltz\nM: 3/4\nL: 1/...,0.853996
9,18478,60382,Dumbarton Castle,James Scott Skinner,https://thesession.org/tunes/18478,march,10,X: 1\nT: Dumbarton Castle\nR: march\nM: 4/4\nL...,0.853894


In [31]:
retriever("test/pointe_au_pic.mp3", limit=10, duration=60, skip_start=5)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,6183,29446,Pointe Au Pic,None,https://thesession.org/tunes/6183,reel,108,X: 2\nT: Pointe Au Pic\nR: reel\nM: 4/4\nL: 1/...,0.937709
1,6183,29445,Pointe Au Pic,None,https://thesession.org/tunes/6183,reel,108,X: 1\nT: Pointe Au Pic\nR: reel\nM: 4/4\nL: 1/...,0.820520
2,3860,41864,The Primrose Polka,None,https://thesession.org/tunes/3860,barndance,72,X: 9\nT: The Primrose Polka\nR: barndance\nM: ...,0.817742
3,10165,11238,The Blue Keys,Brian Finnegan,https://thesession.org/tunes/10165,reel,53,X: 1\nT: The Blue Keys\nR: reel\nM: 4/4\nL: 1/...,0.780700
4,15784,24397,Eklunda,None,https://thesession.org/tunes/15784,waltz,32,X: 1\nT: Eklunda\nR: waltz\nM: 3/4\nL: 1/8\nK:...,0.777654
5,6954,49306,Daisy Bell,Martin Murray,https://thesession.org/tunes/6954,reel,53,X: 2\nT: Daisy Bell\nR: reel\nM: 4/4\nL: 1/8\n...,0.773029
6,2383,47210,Éirinn Go Brách,None,https://thesession.org/tunes/2383,jig,145,X: 4\nT: Éirinn Go Brách\nR: jig\nM: 6/8\nL: 1...,0.770296
7,79,43290,Paddy Ryan’s Dream,None,https://thesession.org/tunes/79,reel,656,X: 5\nT: Paddy Ryan's Dream\nR: reel\nM: 4/4\n...,0.767839
8,6954,49305,Daisy Bell,Martin Murray,https://thesession.org/tunes/6954,reel,53,X: 1\nT: Daisy Bell\nR: reel\nM: 4/4\nL: 1/8\n...,0.762646
9,10177,12923,Fare Well,None,https://thesession.org/tunes/10177,jig,11,X: 2\nT: Fare Well\nR: jig\nM: 6/8\nL: 1/8\nK:...,0.762287


In [34]:
retriever("test/brenda_stubberts.mp3", limit=10, duration=60, skip_start=5)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,1501,36914,The Morpeth Lasses,None,https://thesession.org/tunes/1501,reel,67,X: 10\nT: The Morpeth Lasses\nR: reel\nM: 4/4\...,0.835643
1,727,39040,Brenda Stubbert’s,Jerry Holland,https://thesession.org/tunes/727,reel,1660,X: 2\nT: Brenda Stubbert's\nR: reel\nM: 4/4\nL...,0.824323
2,4249,3533,The Other Clog,None,https://thesession.org/tunes/4249,polka,27,X: 2\nT: The Other Clog\nR: polka\nM: 2/4\nL: ...,0.804148
3,1501,36915,The Morpeth Lasses,None,https://thesession.org/tunes/1501,reel,67,X: 11\nT: The Morpeth Lasses\nR: reel\nM: 4/4\...,0.799995
4,8326,57937,Reel De Montmarquette,None,https://thesession.org/tunes/8326,reel,13,X: 1\nT: Reel De Montmarquette\nR: reel\nM: 4/...,0.784831
5,1501,36913,The Morpeth Lasses,None,https://thesession.org/tunes/1501,reel,67,X: 9\nT: The Morpeth Lasses\nR: reel\nM: 4/4\n...,0.784329
6,1501,36910,The Morpeth Lasses,None,https://thesession.org/tunes/1501,reel,67,X: 6\nT: The Morpeth Lasses\nR: reel\nM: 4/4\n...,0.783790
7,1828,4161,Neil Gow’s Wife,None,https://thesession.org/tunes/1828,strathspey,97,X: 1\nT: Neil Gow's Wife\nR: strathspey\nM: 4/...,0.781465
8,1760,30387,The Curragh Races,None,https://thesession.org/tunes/1760,reel,233,X: 4\nT: The Curragh Races\nR: reel\nM: 4/4\nL...,0.781290
9,708,54095,The Watchmaker,None,https://thesession.org/tunes/708,reel,188,X: 1\nT: The Watchmaker\nR: reel\nM: 4/4\nL: 1...,0.780631


In [37]:
retriever("test/6_8_andre_alain.mp3", limit=10, duration=60, skip_start=5)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,891,52898,The Trip To Durrow,Dan Cleary,https://thesession.org/tunes/891,reel,1065,X: 3\nT: The Trip To Durrow\nR: reel\nM: 4/4\n...,0.876106
1,891,52897,The Trip To Durrow,Dan Cleary,https://thesession.org/tunes/891,reel,1065,X: 2\nT: The Trip To Durrow\nR: reel\nM: 4/4\n...,0.862955
2,203,24036,Frank Roche’s,John Pringle,https://thesession.org/tunes/203,strathspey,121,X: 2\nT: Frank Roche's\nR: strathspey\nM: 4/4\...,0.860670
3,891,52899,The Trip To Durrow,Dan Cleary,https://thesession.org/tunes/891,reel,1065,X: 4\nT: The Trip To Durrow\nR: reel\nM: 4/4\n...,0.836955
4,449,12755,Sí Bheag Sí Mhór,Turlough O’Carolan,https://thesession.org/tunes/449,waltz,2839,X: 12\nT: Sí Bheag Sí Mhór\nR: waltz\nM: 3/4\n...,0.835354
5,13933,51481,The Northern Meeting,None,https://thesession.org/tunes/13933,strathspey,4,X: 1\nT: The Northern Meeting\nR: strathspey\n...,0.817669
6,449,12746,Sí Bheag Sí Mhór,Turlough O’Carolan,https://thesession.org/tunes/449,waltz,2839,X: 3\nT: Sí Bheag Sí Mhór\nR: waltz\nM: 3/4\nL...,0.811290
7,449,12750,Sí Bheag Sí Mhór,Turlough O’Carolan,https://thesession.org/tunes/449,waltz,2839,X: 7\nT: Sí Bheag Sí Mhór\nR: waltz\nM: 3/4\nL...,0.809713
8,449,12759,Sí Bheag Sí Mhór,Turlough O’Carolan,https://thesession.org/tunes/449,waltz,2839,X: 16\nT: Sí Bheag Sí Mhór\nR: waltz\nM: 3/4\n...,0.802966
9,16275,12128,Rose Of Lucerne,None,https://thesession.org/tunes/16275,waltz,4,X: 1\nT: Rose Of Lucerne\nR: waltz\nM: 3/4\nL:...,0.798522


In [42]:
retriever("test/morrissons_jig.mp3", limit=10, duration=60, skip_start=5)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,11420,41911,The Fir Tree,None,https://thesession.org/tunes/11420,strathspey,14,X: 1\nT: The Fir Tree\nR: strathspey\nM: 4/4\n...,0.878689
1,21359,59336,Gan Ainm,None,https://thesession.org/tunes/21359,reel,1,X: 1\nT: Gan Ainm\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.859239
2,71,33981,Morrison’s,None,https://thesession.org/tunes/71,jig,5442,X: 9\nT: Morrison's\nR: jig\nM: 6/8\nL: 1/8\nK...,0.858998
3,1272,59673,The College Groves,None,https://thesession.org/tunes/1272,reel,515,X: 2\nT: The College Groves\nR: reel\nM: 4/4\n...,0.847043
4,9056,4909,Lovely Lough Sheelin,Ed Reavy,https://thesession.org/tunes/9056,strathspey,15,X: 1\nT: Lovely Lough Sheelin\nR: strathspey\n...,0.839042
5,18413,41571,Mr. Abel Bank’s,None,https://thesession.org/tunes/18413,strathspey,3,X: 1\nT: Mr. Abel Bank's\nR: strathspey\nM: 4/...,0.836657
6,500,2316,Rolling In The Barrel,None,https://thesession.org/tunes/500,reel,719,X: 17\nT: Rolling In The Barrel\nR: reel\nM: 4...,0.833737
7,1087,43900,Patsy Touhey’s,None,https://thesession.org/tunes/1087,reel,235,X: 3\nT: Patsy Touhey's\nR: reel\nM: 4/4\nL: 1...,0.825998
8,870,55599,Spórt,Peadar Ó Riada,https://thesession.org/tunes/870,jig,600,X: 8\nT: Spórt\nR: jig\nM: 6/8\nL: 1/8\nK: Dma...,0.823743
9,167,25292,The Dawn,None,https://thesession.org/tunes/167,reel,394,X: 8\nT: The Dawn\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.815161


In [44]:
retriever("test/julia_delaney.mp3", limit=10, duration=60, skip_start=5)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,19640,61276,The Black Knife,None,https://thesession.org/tunes/19640,jig,13,X: 2\nT: The Black Knife\nR: jig\nM: 6/8\nL: 1...,0.908940
1,23674,4694,Mr. Malone,Turlough O’Carolan,https://thesession.org/tunes/23674,waltz,5,X: 1\nT: Mr. Malone\nR: waltz\nM: 3/4\nL: 1/8\...,0.871084
2,5787,23434,Da Silver Bow,None,https://thesession.org/tunes/5787,polka,43,X: 1\nT: Da Silver Bow\nR: polka\nM: 2/4\nL: 1...,0.838979
3,19931,36202,Jemmy Duffy’s,None,https://thesession.org/tunes/19931,barndance,2,X: 1\nT: Jemmy Duffy's\nR: barndance\nM: 4/4\n...,0.831888
4,18453,29952,Deuks Dang Ower My Daddy,William McGibbon,https://thesession.org/tunes/18453,jig,4,X: 3\nT: Deuks Dang Ower My Daddy\nR: jig\nM: ...,0.828814
5,11648,21340,Failte Na Miosg,None,https://thesession.org/tunes/11648,waltz,11,X: 1\nT: Failte Na Miosg\nR: waltz\nM: 3/4\nL:...,0.828366
6,20079,26359,The Maids Of Ballinacarty,None,https://thesession.org/tunes/20079,jig,2,X: 1\nT: The Maids Of Ballinacarty\nR: jig\nM:...,0.823711
7,1574,18303,The Boogie,John Nolan,https://thesession.org/tunes/1574,reel,100,X: 2\nT: The Boogie\nR: reel\nM: 4/4\nL: 1/8\n...,0.822322
8,589,34420,Julia Delaney’s,None,https://thesession.org/tunes/589,reel,1806,X: 6\nT: Julia Delaney's\nR: reel\nM: 4/4\nL: ...,0.817139
9,20412,15399,Ballinakill,None,https://thesession.org/tunes/20412,polka,2,X: 1\nT: Ballinakill\nR: polka\nM: 2/4\nL: 1/8...,0.809988
